In [1]:
from utils import *
from models.continuous_policy import Policy
from models.critic import Value
from models.discrete_policy import DiscretePolicy

from algos.mujocoEnv import MujocoEnv
import pickle, time
# PATH = "C:/Users/Kartik/My Drive (kartiknagpal@berkeley.edu)/ResearchCode/graph-transformer-tamp/baselines/PyTorch-CPO/assets/learned_models/CPO/Mujoco-Env/2024-02-02-exp-2-mujocoEnv/model.p"

env = MujocoEnv()

# checkpoint = torch.load("./CPO-Best.pt")
# print(checkpoint)

policy, value, running_state = pickle.load(open("./assets/learned_models/CPO/Mujoco-Env/2024-02-02-exp-5-mujocoEnv/model.p", "rb"))
# baselines\PyTorch-CPO\assets\learned_models\CPO\Mujoco-Env\2024-02-02-exp-2-mujocoEnv\model.p
# policy.to(device)
# value.to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


c:\Users\Kartik\anaconda3\envs\srlnbc\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


cpu


In [ ]:
while num_steps < min_batch_size:
    state = env.reset()
    if running_state is not None:
        state = running_state(state)
    reward_episode = 0
    env_reward_episode = 0
    reward_episode_list_1 = []
    env_reward_episode_list_1 = []
    episode_constraint_respect = True
    episode_completed = False
    
    for t in range(1):
        state_var = tensor(state).unsqueeze(0)
        with torch.no_grad():
            if mean_action:
                action = policy(state_var)[0][0].numpy()
            else:
                action = policy.select_action(state_var)[0].numpy()
        action = action.astype(np.float64)
        next_state, reward, done, respect = env.step(action)
        env.viewer.sync()
        env_reward_episode += reward
        env_reward_episode_list_1.append(reward)

        if running_state is not None:
            next_state = running_state(next_state)

        if not respect:
            episode_constraint_respect = False

        if done:
            episode_completed = True
            break

        state = next_state

In [2]:
import scipy.stats

num_episodes = 10
eval_stats = np.zeros((num_episodes, 3))
# policy = policy_net

# NOT SURE
mean_action = True


for episode in range(num_episodes):
    state = env.reset()
    if running_state is not None:
        state = running_state(state)
    reward_episode = 0
    env_reward_episode = 0
    reward_episode_list_1 = []
    env_reward_episode_list_1 = []
    episode_constraint_respect = True
    episode_completed = False
    print("Episode: ", episode)

    for t in range(500):
        state_var = state.unsqueeze(0)
        with torch.no_grad():
            if mean_action:
                action = policy(state_var)[0][0].numpy()
            else:
                action = policy.select_action(state_var)[0].numpy()
        action = np.nan_to_num(action.astype(np.float64))
        next_state, reward, done, respect = env.step(action)
        
        # print(env.getJoints())
        env.viewer.sync()
        time.sleep(0.1)
        env_reward_episode += reward
        env_reward_episode_list_1.append(reward)

        if running_state is not None:
            next_state = running_state(next_state)

        if not respect:
            episode_constraint_respect = False

        if done:
            episode_completed = True
            break

        state = next_state

    eval_stats[episode] = [env_reward_episode.cpu(), episode_constraint_respect, episode_completed]

def get_confidence_interval(data, confidence=0.95):
    m, se = np.mean(data), scipy.stats.sem(data)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., len(data)-1)
    return m, m-h, m+h, h


print("Reward CI:" + str(get_confidence_interval(eval_stats[:,0])))
print("Respect CI:" + str(get_confidence_interval(eval_stats[:,1])))
print("Percent Completion:" + str(100*sum(eval_stats[:,2])/num_episodes))
eval_stats[eval_stats[:,1] == 0.0, 2] = 0.0
print("Percent Completion w/o Violation:" + str(100*sum(eval_stats[:,2])/num_episodes))

# print(f"Reward CI: {get_confidence_interval(eval_stats[:,0])} /t Average respect: {get_confidence_interval(eval_stats[:,1])} \t Percent Completion: {100*sum(eval_stats[:,2])/num_episodes} %")


Episode:  0


c:\Users\Kartik\My Drive (kartiknagpal@berkeley.edu)\ResearchCode\graph-transformer-tamp\baselines\PyTorch-CPO\algos\mujocoEnv.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_state = torch.tensor(new_state, device=device)


Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
Reward CI:(-258.0754684448242, -298.93770895273593, -217.21322793691246, 40.86224050791172)
Respect CI:(0.6, 0.230591282220163, 0.969408717779837, 0.36940871777983697)
Percent Completion:0.0
Percent Completion w/o Violation:0.0
